# Worksheet 08

Name:  Prathmesh Sonawane

UID: U39215370

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [78]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.9842549899902515, 3.8422770690967702, 3.8053064870179276, 5.170320918484939, 6.065765817286388, 3.90825732996152, 3.9058540115629574, 3.5286606066705626, 3.7040700613908597, 5.1997574580661725]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [77]:
s2 = np.random.normal( 8 , 1 , 10).tolist()
print(s2)

[8.36142819099992, 7.201808006039365, 7.989662368213676, 8.720887687882268, 6.625200072160831, 8.035787240240781, 7.605042303760434, 8.227109386977165, 7.9552634456075175, 8.017259884699836]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [79]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1[len(s1)-1]
        s1.remove(p1)
        data.append(p1)
    else:
        p2 = s2[len(s2)-1]
        s2.remove(p2)
        data.append(p2)
print(data)

[5.1997574580661725, 3.7040700613908597, 3.5286606066705626, 8.017259884699836, 3.9058540115629574, 3.90825732996152, 6.065765817286388, 5.170320918484939, 3.8053064870179276, 7.9552634456075175]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Parameters of the GMM: 

Species One :

  -Mean: 5

  -Variance: 1


Species Two: 

  -Mean: 8

  -Variance: 1
  
Probability of a given point being Species 1: 0.5 

Probability of a given point being Species 2: 0.5 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [80]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 =  [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2))]
mean = [ sum(s1)/len(s1) ,  sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2))  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.1997574580661725, 3.7040700613908597, 3.5286606066705626, 3.9058540115629574, 3.90825732996152, 5.170320918484939, 3.8053064870179276]
[8.017259884699836, 6.065765817286388, 7.9552634456075175]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 4.174603839022135,  mean_2 = 7.346096382531247
var_1 = 0.4229406581616018,  var_2 = 2.460791313680396


/Users/prathmesh/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


This analysis is relatively close to some of our values. 

Though we have a 50 percent chance of getting a point from either cluster, in our analysis, we find that almost 0.70 percent of points belong to one, with the rest belonging to another.

In addition, the estimated means of our data points in the cluster are 4.17 and 7.34, which are close to 5 and 8 respectively. 

The variances we estimated are also 0.423 and 2.46 which are not close to initial variances of 1. This could be due to the fact that we have a small sample size. 

In total, though the estimations are close, they could be better. However, as is, they provide us with a good idea of how some of our clusters look. 


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [91]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

print(data)
for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] *  prob_s[0] / prob_x )
    prob_s1_x.append(pdf_i[1] *  prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


[5.1997574580661725, 3.7040700613908597, 3.5286606066705626, 8.017259884699836, 3.9058540115629574, 3.90825732996152, 6.065765817286388, 5.170320918484939, 3.8053064870179276, 7.9552634456075175]
point =  5.1997574580661725
probability of observing that point if it came from cluster 0 =  0.17274254393755506
probability of observing that point if it came from cluster 1 =  0.07966396048490827
point =  3.7040700613908597
probability of observing that point if it came from cluster 0 =  0.4788329070083308
probability of observing that point if it came from cluster 1 =  0.04493324764987685
point =  3.5286606066705626
probability of observing that point if it came from cluster 0 =  0.3868821148939514
probability of observing that point if it came from cluster 1 =  0.04139590488232727
point =  8.017259884699836
probability of observing that point if it came from cluster 0 =  2.577661939312275e-08
probability of observing that point if it came from cluster 1 =  0.12663932722344662
point =  3.90

Our list of points that this program recieves is:
 [5.1997574580661725, 3.7040700613908597, 3.5286606066705626, 8.017259884699836, 3.9058540115629574, 3.90825732996152, 6.065765817286388, 5.170320918484939, 3.8053064870179276, 7.9552634456075175]

If the cluster with mean 5 is named "Cluster One" and the cluster with mean 8 is named "Cluster Two", then each point in our above list can be mapped to this where each indice maps a point to a cluster: 

["Cluster One", "Cluster One", "Cluster One", "Cluster Two", "Cluster One", "Cluster One", "Cluster One", "Cluster One", "Cluster One", "Cluster Two"]

This was found by purely looking at the probabilities and assigning them accordingly. I would say that it compares pretty closely to the truth. The only point that did not have a very high probability (like 90% or more) of being from one cluster or another was point "6.065765817286388", but I would say that it is still accurate. 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [96]:
prob_c = [sum(prob_s0_x)/len(prob_s0_x), sum(prob_s1_x)/len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 4.189019877649082,  mean_2 = 7.001339271108808
var_1 = 0.485560967038552,  var_2 = 1.7209864363814171


Firstly, the proportion of points from each cluster stayed the same. 

In addition, both means here are slighly lower than the means calculated for both cluster using k-means. 

Lastly, both variances are lower than those found using k-means. However, The variance for cluster 2 almost 0.7 value lower than that estimated from k-means. 

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [97]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        # pdf_i.append(norm.pdf(p, mean[j], var[j]))
        pdf_i.append(norm.pdf(p, mean[j], var[j]**0.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0]*prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1]*prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  5.1997574580661725
probability of observing that point if it came from cluster 0 =  0.1999510651452784
probability of observing that point if it came from cluster 1 =  0.11843830062679635
point =  3.7040700613908597
probability of observing that point if it came from cluster 0 =  0.4493815212115692
probability of observing that point if it came from cluster 1 =  0.01291925595170025
point =  3.5286606066705626
probability of observing that point if it came from cluster 0 =  0.3654027358647048
probability of observing that point if it came from cluster 1 =  0.009149594357119822
point =  8.017259884699836
probability of observing that point if it came from cluster 0 =  1.598643296244019e-07
probability of observing that point if it came from cluster 1 =  0.22531791731408887
point =  3.9058540115629574
probability of observing that point if it came from cluster 0 =  0.5271444797321052
probability of observing that point if it came from cluster 1 =  0.018793122220163334
point =  3.

After reassigning these values, our probabilities for whether a point belongs to one or another cluster have become far closer to either 1 or 0, showing that our certainty has increased. Thus, we can better allocate points to one or the other cluster. 

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [99]:
probs = zip(data, prob_s0_x, prob_s1_x)
assignment = zip(data, [0 if p[1] > p[2] else 1 for p in probs])
print([[p, i] for p, i in assignment])

print("Above, 0 stands for the cluster with mean 5 and 1 stands for the cluster with mean 8")

[[5.1997574580661725, 0], [3.7040700613908597, 0], [3.5286606066705626, 0], [8.017259884699836, 1], [3.9058540115629574, 0], [3.90825732996152, 0], [6.065765817286388, 1], [5.170320918484939, 0], [3.8053064870179276, 0], [7.9552634456075175, 1]]
Above, 0 stands for the cluster with mean 5 and 1 stands for the cluster with mean 8
